In [71]:
# Install dependencies
import sys
!{sys.executable} -m pip install numpy matplotlib pandas openpyxl sklearn seaborn matplotlib


[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: C:\Users\natha\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [72]:
# Imports and global options
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import multiprocessing
from functools import partial

In [73]:
# prepare dataset
def prepare_dataset(dataset_file, input_cols, result_col=None):
    # Load data
    df = pd.read_excel(dataset_file)
    
    # Filter bad data, replace categories with dummies
    x_df = df[input_cols].apply(lambda x: x.str.strip())
    x = pd.get_dummies(x_df, drop_first=True)
    
    if result_col:
        y_df = df[result_col].str.strip()
        y = pd.get_dummies(y_df, drop_first=True).iloc[:, 0]
        
        return x, y
    else:
        return x

In [74]:
# Data and dummies displayed in a table
def display_input_data(dataset_file, input_cols):
    x = prepare_dataset(dataset_file, input_cols)
    display(x)

In [ ]:
def check_input_data_indices(train_dataset_file, test_dataset_file, input_cols):
    x_train = prepare_dataset(train_dataset_file, input_cols)
    x_test = prepare_dataset(test_dataset_file, input_cols)
    
    if not x_train.columns.equals(x_test.columns):
        raise Exception('Datasets are incompatible!');
    print('Datasets are compatible')

In [75]:
def train_model(train_dataset_file, input_cols, result_col):
    x, y = prepare_dataset(train_dataset_file, input_cols, result_col)
    
    # Instantiate the model
    model = LogisticRegression(solver='liblinear', random_state=0)

    # Train (=fit) the model
    model.fit(x, y)
    
    return model


In [76]:
def evaluate_model(model, test_dataset_file, input_cols, result_col):
    x, y = prepare_dataset(test_dataset_file, input_cols, result_col)
    
    return model.score(x, y)

In [2]:
def display_correlation_matrix(dataset_file, input_cols):
    x = prepare_dataset(dataset_file, input_cols)

    plt.figure(figsize=(15,10))
    sns.heatmap(x.corr(method='pearson'), annot=True)

In [ ]:
def _evaluate_for_input_combination(train_dataset_file, test_dataset_file, input_combination, result_col):
    #print(input_cols_comb)
    model = train_model(
        train_dataset_file,
        list(input_combination),
        result_col
    )
    score = evaluate_model(
        model,
        test_dataset_file,
        list(input_combination),
        result_col
    )
    #print(score)
    return score

def run_input_optimizer(train_dataset_file, test_dataset_file, input_cols, result_col):
    # Contains all combinations by input length (3D list)
    input_combinations = []
    for l in range(2, len(input_cols) + 1):
        input_combinations.append(list(itertools.combinations(input_cols, l)))

    scores = []
    flat_input_combinations = []
    
    for input_combinations_of_len in input_combinations:
        print('Testing combinations of length ' + str(len(input_combinations_of_len[0])))
        
        for input_combination in input_combinations_of_len:
            score = _evaluate_for_input_combination(train_dataset_file, test_dataset_file, input_combination, result_col)
            scores.append(score)
            flat_input_combinations.append(input_combination)

    # Pick and print results
    max_score = max(scores)
    max_score_idx = scores.index(max_score)
    max_score_cols = flat_input_combinations[max_score_idx]

    min_score = min(scores)
    min_score_idx = scores.index(min_score)
    min_score_cols = flat_input_combinations[min_score_idx]

    print('Max score: ' + str(max_score))
    print('Best columns combination: ' + str(max_score_cols))
    print('Min score: ' + str(min_score))
    print('Worse columns combination: ' + str(min_score_cols))